In [2]:
import numpy as np
import cv2
import glob
import tensorflow as tf
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
import pickle

import os
import pathlib
import time
import datetime

from matplotlib import pyplot as plt
from IPython import display

In [3]:
path_file = 'archive/*png'

52001


In [103]:
# We will pixelate the image for faces
def load_image_train(image_file):
    # Read and decode an image file to a uint8 tensor
    image = tf.io.read_file(image_file)
    image = tf.io.decode_jpeg(image)

    # Convert images to float32 tensors
    input_image = tf.cast(image, tf.float32)
    input_image = tf.image.resize(input_image, [IMG_HEIGHT, IMG_WIDTH],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    input_image = tf.cast(input_image, tf.float32) / 255.0 
    
    #Obtain pixelated image
    width = int(input_image.shape[1] * scale_percent / 100)
    height = int(input_image.shape[0] * scale_percent / 100)
    
    small_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.AREA)

    pixelated_image = tf.image.resize(small_image, [IMG_HEIGHT, IMG_WIDTH],
                                method=tf.image.ResizeMethod.AREA)
   

    return input_image, pixelated_image

In [38]:
def make_data(global_batch):
    BUFFER_SIZE = 400
    # The batch size of 1 produced better results for the U-Net in the original pix2pix experiment
    BATCH_SIZE = 1
    # Each image is 256x256 in size
    IMG_WIDTH = 200
    IMG_HEIGHT = 200
    scale_percent = 30
#     path = "BD_project/lfw/*/*.jpg"
    path = 'archive/*.png'

    
    # We will pixelate the image for faces
    def load_image_train(image_file):
        # Read and decode an image file to a uint8 tensor
        image = tf.io.read_file(image_file)
        image = tf.io.decode_jpeg(image)

        # Convert images to float32 tensors
        input_image = tf.cast(image, tf.float32)
        input_image = tf.image.resize(input_image, [IMG_HEIGHT, IMG_WIDTH],
                                    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        input_image = tf.cast(input_image, tf.float32) / 255.0 

        #Obtain pixelated image
        width = int(input_image.shape[1] * scale_percent / 100)
        height = int(input_image.shape[0] * scale_percent / 100)

        small_image = tf.image.resize(input_image, [height, width],
                                    method=tf.image.ResizeMethod.AREA)

        pixelated_image = tf.image.resize(small_image, [IMG_HEIGHT, IMG_WIDTH],
                                    method=tf.image.ResizeMethod.AREA)


        return input_image, pixelated_image
    
    train_dataset = tf.data.Dataset.list_files(path)
    train_dataset = train_dataset.map(load_image_train,
                                      num_parallel_calls=tf.data.AUTOTUNE)
    train_dataset = train_dataset.shuffle(BUFFER_SIZE)
    train_dataset = train_dataset.batch(global_batch)
    
    return train_dataset

In [39]:
from enum import auto
def build_and_compile_latent_model():
    #with strategy.scope():
    Input_img = Input(shape=(None, None, 3))  

    #encoding architecture
    x1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(Input_img)
    x2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x1)
    x3 = MaxPool2D(padding='same')(x2)

    x4 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x3)
    x5 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x4)
    x6 = MaxPool2D(padding='same')(x5)

    encoded = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x6)
    #encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

    # decoding architecture
    x7 = UpSampling2D()(encoded)
    x8 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x7)
    x9 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x8)
    x10 = Add()([x5, x9])

    x11 = UpSampling2D()(x10)
    x12 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x11)
    x13 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x12)
    x14 = Add()([x2, x13])

    # x3 = UpSampling2D((2, 2))(x3)
    # x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
    # x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
    decoded = Conv2D(1, (3, 3), padding='same',activation='relu', kernel_regularizer=regularizers.l1(10e-10))(x14)

    autoencoder = Model(Input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

    return autoencoder

In [40]:
import json
import os

def train_func_distributed():
    per_worker_batch_size = 8
    # This environment variable will be set by Ray Train.
    tf_config = json.loads(os.environ['TF_CONFIG'])
    num_workers = len(tf_config['cluster']['worker'])

    strategy = tf.distribute.MultiWorkerMirroredStrategy()

    global_batch_size = per_worker_batch_size * num_workers
    multi_worker_dataset = make_data(global_batch_size)

    with strategy.scope():
        # Model building/compiling need to be within `strategy.scope()`.
        multi_worker_model = build_and_compile_latent_model()

    multi_worker_model.fit(multi_worker_dataset, epochs=3)



In [41]:
from ray.train import Trainer

trainer = Trainer(backend="tensorflow", num_workers=1)

# For GPU Training, set `use_gpu` to True.
# trainer = Trainer(backend="tensorflow", num_workers=4, use_gpu=True)

trainer.start()
results = trainer.run(train_func_distributed)
trainer.shutdown()


2022-04-08 14:39:02,967	INFO trainer.py:199 -- Trainer logs will be logged in: /Users/vaibhavsingh/ray_results/train_2022-04-08_14-39-02
2022-04-08 14:39:03,902	INFO trainer.py:205 -- Run results will be logged in: /Users/vaibhavsingh/ray_results/train_2022-04-08_14-39-02/run_001
(BaseWorkerMixin pid=20621) 2022-04-08 14:39:10.632824: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(BaseWorkerMixin pid=20621) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(BaseWorkerMixin pid=20621) 2022-04-08 14:39:10.636237: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 127.0.0.1:56763}
(BaseWorkerMixin pid=20621) 2022-04-08 14:39:10.636359: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:437] Started ser

(BaseWorkerMixin pid=20621) Epoch 1/3
  15/6501 [..............................] - ETA: 6:15:31 - loss: 0.0390 - accuracy: 0.0012


KeyboardInterrupt: 